In [1]:
import pandas as pd

reviews_raw: pd.DataFrame = pd.read_csv("./data/letterboxd_250movie_reviews.csv")

reviews_raw.head()
valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)
valid_reviews.count()

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_87962/4114284106.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_87962/4114284106.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)


Unnamed: 0    61096
Review        61096
Rating        61096
Date          61096
Status        61096
Movie         61096
dtype: int64

In [2]:
valid_reviews.head()

,Unnamed: 0,Review,Rating,Date,Status,Movie
43340,43340,A scathing political thriller from the great C...,★★★★★,19 Jul 2022,Watched,z
29334,29334,It's impossible to not respect how kids of a c...,★★★★½,04 Aug 2021,Watched,where-is-the-friends-house
72718,72718,"""There is only one golden rule: ""Do unto other...",★★★★★,21 Oct 2023,Watched,eureka
87904,87904,This is what I expect from screenplay when I s...,★★★★★,02 Jan 2022,Watched,happy-together-1997
57955,57955,"#madlad Mifune'If you would shed blood, then l...",★★★★,13 Apr 2021,Watched,throne-of-blood


remove non-english reviews:

In [3]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistent results
DetectorFactory.seed = 0

def is_english(text: str) -> bool:
    try:
        return detect(text) == 'en'
    except LangDetectException:
        # Handle very short texts or detection failures
        return False

is_english(valid_reviews['Review'].iloc[0])
valid_reviews = valid_reviews[valid_reviews['Review'].apply(is_english)]

sample 200 reviews for each rating:

In [4]:
sample_for_each_rating = 200
sampled_valid_reviews = pd.concat([valid_reviews[valid_reviews['Rating'] == rating].sample(n = sample_for_each_rating, random_state=42) for rating in valid_reviews['Rating'].unique()])
sampled_valid_reviews = sampled_valid_reviews.sample(frac=1, random_state=42) # shuffle again to avoid ordered by rating
sampled_valid_reviews = sampled_valid_reviews.drop(columns=['Status','Date']).rename(columns={'Unnamed: 0': 'Original_ID'})
sampled_valid_reviews.head()

,Original_ID,Review,Rating,Movie
3029,3029,Chaaarrrrrlieeee wtf man this bad :/Itll be a ...,★,parasite
18442,18442,“I can't waste time wondering if I made mistak...,★★★★½,in-the-mood-for-love
77401,77401,Life has a tendency to feel like a series of t...,★★★,aparajito
68592,68592,im a little bit confused but i've got the spir...,★★,mulholland-drive
38793,38793,A friend of mine once said:“Films are a joy. T...,★★★,the-wages-of-fear


see some of the reviews:

In [5]:
def summarize(df_row)-> str:
    return f"Review: {df_row['Review']} - Movie: {df_row['Movie']}"

def check_rating(rating):
    return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()

def log_sample_for_rating(rating):
    samples = check_rating(rating)
    print(f"Samples for rating {rating}:")
    for sample in samples:
        print(f' {sample}')
    print("\n")
log_sample_for_rating('★★★★★')
log_sample_for_rating('★★½')
log_sample_for_rating('½')

Samples for rating ★★★★★:
 Review: First time on a big screen and I'm literally overwhelmed: after almost thirty years Satoshi Kon is still beyond the beyond. - Movie: perfect-blue
 Review: That thing going around on TikTok that’s like “what’s an acting performance that was SO GOOD that you forgot it was acting??” and it’s just this in its entirety - Movie: whos-afraid-of-virginia-woolf
 Review: An advertisement of silence from god in a land of souls without eyelids. - Movie: werckmeister-harmonies
 Review: bob fosse was a human bojack horseman - Movie: all-that-jazz
 Review: About to lose my fucking minddddddddd - Movie: opening-night


Samples for rating ★★½:
 Review: Halfway through this I hit the play button on accident which caused WAP to start playing and honestly? Heightened the experience. - Movie: lawrence-of-arabia
 Review: "I had to execute someone, and there is something about it I didn't like." -Lawrence, Four hours of colonialism, sand, brown face and boredom.. not my thi

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_87962/4154530477.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_87962/4154530477.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_87962/4154530477.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()


In [7]:
# save the sampled reviews to a new CSV file
sampled_valid_reviews.to_csv('./letterboxd_250movie_reviews_normalized_sampled.csv', index=False)

# divide into training, validation, and test sets
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1
train_size = int(len(sampled_valid_reviews) * train_frac)
val_size = int(len(sampled_valid_reviews) * val_frac)   
test_size = int(len(sampled_valid_reviews) * test_frac)
train_set = sampled_valid_reviews[:train_size]
val_set = sampled_valid_reviews[train_size:train_size + val_size]
test_set = sampled_valid_reviews[train_size + val_size:]
print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")
train_set.to_csv('./data/letterboxd_250movie_reviews_train.csv', index=False)
val_set.to_csv('./data/letterboxd_250movie_reviews_val.csv', index=False)
test_set.to_csv('./data/letterboxd_250movie_reviews_test.csv', index=False)

Train set size: 1600
Validation set size: 200
Test set size: 200
